# JijZept Solverを用いたハンズオン

## パッケージのインストール
まず、このノートブックに必要なパッケージをインストールしましょう。

In [ ]:
# 必要なパッケージをインストール
!pip install jijzept_solver jijmodeling ommx_pyscipopt_adapter

# JijZept Solver 環境設定

In [ ]:
import os

os.environ["JIJZEPT_SOLVER_SERVER_HOST"] = "APIサーバーのホスト名"
os.environ["JIJZEPT_SOLVER_ACCESS_TOKEN"] = "アクセストークン"

# 簡略化されたジョブスケジューリング問題の解説

## 1. はじめに：スケジューリングの課題

スケジューリング問題は、多くの産業や実世界のシナリオで一般的です。一般的な目標は、タスク（または「ジョブ」）を利用可能なリソース（「機械」や作業者など）に時間的に割り当て、様々な制約を遵守しながら、特定の目的（コストや完了時間の最小化など）を達成することです。

これらの問題は以下の理由により、すぐに非常に複雑になる可能性があります：

* **組み合わせ的性質：** 可能なスケジュールの数が膨大になることがよくあります。
* **制約：** リソースの可用性、タスクの依存関係（優先順位）、締切、セットアップ時間などのルールが難易度の層を追加します。

例としては、工場の生産計画、プロジェクトタスク管理、物流ルーティング、オペレーティングシステムのタスクスケジューリングなどがあります。数理最適化は、これらの困難な問題に対して最適または近似最適な解を見つけるための強力なフレームワークを提供します。

## 2. 焦点：並列機械スケジューリング問題（PMSP）

このノートブックでは、**並列機械スケジューリング問題（PMSP）** として知られる、基本的で広く適用可能なタイプのスケジューリング問題に焦点を当てます。

**PMSPのセットアップ：**

* 複数の独立した**ジョブ**があります。各ジョブは実行する必要がある単一のタスクを表します。
* 並列に動作する複数の同一（または関連）**機械**があります。
* 各ジョブ `i` には、どの機械で実行されるかに関係なく、完了するために必要な特定の**処理時間**（`JT[i]`）があります。
* 各ジョブは**正確に1つ**の機械に割り当てる必要があります。
* 機械は複数のジョブを処理できますが、**一度に1つのジョブのみ**処理できます。

**目標（目的）：**

私たちの主な目的は**メイクスパンを最小化する**ことです。

* **メイクスパン：** *すべて*の機械にわたって*最後*のジョブが終了する時間。これは、割り当てられたすべてのジョブを完了するのに最も長くかかる機械によって決定されます。
* **目的：** 最も忙しい機械の完了時間が最小化されるようなジョブから機械への割り当てを見つけます。これにより、利用可能な機械全体でワークロードが効果的にバランスされます。

このノートブックでは、JijModelingとJijZept Solverを使用して、このPMSPをモデル化・求解する方法を示します。

## 3. 問題の構成要素（PMSP）

PMSPモデルの主要な要素を定義しましょう：

* **ジョブ：** スケジュールされる独立したタスクのセット（インデックス `i = 0, 1, ..., N-1`）。
* **機械：** ジョブを処理するために利用可能な並列リソースのセット（インデックス `m = 0, 1, ..., M-1`）。
* **処理時間（`JT[i]`）：** 任意の機械でジョブ `i` を完了するために必要な時間。

## 4. 目的関数（メイクスパンの最小化）

前述のように、目標はメイクスパンを最小化することです。これは、すべての機械の中で最大の完了時間です。

## 5. 主要な制約（PMSP）

有効なスケジュールを確保するために、2つの主要なタイプの制約が必要です：

1.  **ジョブ割り当て制約：** すべてのジョブは正確に1つの機械に割り当てる必要があります。
    * 数学的に：各ジョブ `i` について、すべての機械 `m` にわたる割り当て `x[i, m]` の合計は 1 に等しくなければなりません。
2.  **メイクスパン制約：** メイクスパンは、*任意の*機械 `m` の総処理時間（負荷）以上でなければなりません。
    * 数学的に：各機械 `m` について、割り当てられたすべてのジョブ `i` の処理時間 `JT[i]` の合計（`JT[i] * x[i, m]`）は、`makespan` 変数以下でなければなりません。

これらの制約の下で `makespan` 変数を最小化することにより、オプティマイザは負荷をバランスし、最大負荷を最小化する割り当てを見つけることを余儀なくされます。

## 6. 数理モデルの定義（JijModeling）

それでは、JijModelingを使用してこのPMSPを表現しましょう。

In [ ]:
# 必要なライブラリをインポート
import jijmodeling as jm

# --- 1. プレースホルダ（入力データ） ---
# JT[i]: ジョブiの処理時間
JT = jm.Placeholder("JT", ndim=1, description="ジョブiの処理時間")
# N: ジョブ数（JTの長さから導出）
N = JT.len_at(0, latex="N", description="ジョブ数")
# M: 機械数
M = jm.Placeholder("M", description="機械数")

# --- 2. 要素（インデックス） ---
# i: ジョブのインデックス（0からN-1）
i = jm.Element("i", belong_to=(0, N), description="ジョブのインデックス")
# m: 機械のインデックス（0からM-1）
m = jm.Element("m", belong_to=(0, M), description="機械のインデックス")

# --- 3. 変数（決定変数と補助変数） ---
# x[i, m]: バイナリ変数、ジョブiが機械mに割り当てられる場合は1、そうでない場合は0
x = jm.BinaryVar(
    "x",
    shape=(N, M),
    description="x[i, m]: ジョブiが機械mに割り当てられる場合は1、そうでない場合は0",
)
# makespan: すべての機械にわたる最大完了時間を表す連続変数
makespan = jm.ContinuousVar(
    "makespan",
    lower_bound=0,
    upper_bound=jm.sum(i, JT[i]),
    description="メイクスパン：すべての機械にわたる最後のジョブの完了時間",
)

# --- 4. 問題の定義 ---
problem = jm.Problem("並列機械スケジューリング")

# --- 5. 制約 ---
# 制約1：各ジョブ 'i' は正確に1つの機械 'm' に割り当てられなければならない
problem += jm.Constraint(
    "ジョブ割り当て",
    jm.sum(m, x[i, m]) == 1,
    forall=i,
)
# 制約2：任意の機械 'm' の総処理時間はメイクスパン以下でなければならない
problem += jm.Constraint(
    "機械ごとの最大時間",
    jm.sum(i, JT[i] * x[i, m]) <= makespan,
    forall=m,
)

# --- 6. 目的関数 ---
# メイクスパンを最小化
problem += makespan

problem

上記のコードは、並列機械スケジューリング問題の数理モデルを定義しています。

* `x[i, m]` はコアとなる**決定**を表します：どの機械がどのジョブを取得するか。
* `makespan` は最小化したい全体的なスケジュールの**パフォーマンス**（完了時間）を表します。
* `Job_Assignment` 制約は、すべてのジョブが処理されることを保証します。
* `Max_Time_per_Machine` 制約は、割り当てをメイクスパンにリンクし、メイクスパンが最も忙しい機械に対応できるほど大きいことを保証します。`makespan` を最小化すると、暗黙的に負荷がバランスされます。

## 7. 結果解析と可視化関数

結果の解析と可視化を再利用可能にするため、関数として定義します。

In [ ]:
# 必要に応じてプロットライブラリをインポート
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd  # DataFrameの表示用

def analyze_schedule_results(solution, instance_data):
    x_result: dict[tuple[int, ...], float] = solution.extract_decision_variables("x")
    
    makespan_value = solution.objective
    print(f"\n--- Solver Results ---")
    print(f"Makespan: {makespan_value:.2f}")
    assignment = {}  # {機械インデックス: [ジョブインデックス, ...]}
    assigned_jobs_flat = (
        []
    )  # DataFrameに格納するデータ: [{'Job': i, 'Machine': m, 'Time': t}]
    x_result = {}

    x_result = solution.extract_decision_variables("x")

    # 抽出したx_result辞書を処理
    num_machines_instance = instance_data["M"]
    for indices, val in x_result.items():
        # indicesは(i, m)のようなタプルであるべき
        if len(indices) == 2 and val > 0.5:  # バイナリ '1' をチェック
            i, m = indices  # インデックスをアンパック
            if m not in assignment:
                assignment[m] = []
            assignment[m].append(i)
            # JTにアクセスする前にジョブインデックスiが有効であることを確認
            if 0 <= i < len(instance_data["JT"]):
                assigned_jobs_flat.append(
                    {"Job": i, "Machine": m, "Time": instance_data["JT"][i]}
                )
            else:
                print(
                    f"Warning: Invalid job index {i} found in solution variable 'x' for machine {m}."
                )

    print("\nJob Assignments per Machine:")
    if assignment:
        for m_idx in range(num_machines_instance):
            assigned_jobs = sorted(assignment.get(m_idx, []))
            print(f"  Machine {m_idx}: Jobs {assigned_jobs}")
    else:
        # 解が得られたにもかかわらずx_result抽出が失敗または空の場合に発生する可能性がある
        print("  No assignments could be extracted from 'x' variables.")

    # Pandas DataFrameを使用して割り当てを明確に表示
    if assigned_jobs_flat:
        assignment_df = pd.DataFrame(assigned_jobs_flat)
        print("\nAssignment Details (DataFrame):")
        print(
            assignment_df.sort_values(by=["Machine", "Job"]).reset_index(
                drop=True
            )
        )
    else:
        print("\nAssignment Details (DataFrame): Empty")
        assignment_df = pd.DataFrame()
    
    # --- 解析: 機械負荷の計算 ---
    print("\n--- Result Analysis ---")
    print("Total Processing Time (Load) per Machine:")
    machine_loads = {m: 0 for m in range(num_machines_instance)}
    machine_load_list = []  # プロットと検証用に負荷を格納
    job_times = instance_data["JT"]  # 便宜上のエイリアス

    # 'assignment' 辞書から直接負荷を計算
    for m_idx in range(num_machines_instance):
        load = 0
        assigned_job_indices = assignment.get(m_idx, [])
        for job_idx in assigned_job_indices:
            if 0 <= job_idx < len(job_times):
                load += job_times[job_idx]
            else:
                print(
                    f"Warning: Invalid job index {job_idx} found for machine {m_idx} during load calculation."
                )
        machine_loads[m_idx] = load
        machine_load_list.append(load)
        print(f"  Machine {m_idx}: {load:.2f}")

    # --- 検証 ---
    calculated_max_load = max(machine_load_list) if machine_load_list else 0
    print(f"\nCalculated Maximum Machine Load: {calculated_max_load:.2f}")
    print(f"(Solver's Makespan: {makespan_value:.2f})")
    # 浮動小数点比較のための小さな許容誤差を使用
    if abs(calculated_max_load - makespan_value) < 1e-6:
        print("-> Verification successful: Makespan matches the maximum machine load.")
    else:
        print(
            "-> WARNING: Makespan does NOT match the calculated maximum load. Check model/solver."
        )
    
    return {
        'makespan_value': makespan_value,
        'assignment': assignment,
        'machine_loads': machine_loads,
        'machine_load_list': machine_load_list,
        'assignment_df': assignment_df,
        'job_times': job_times,
        'num_machines': num_machines_instance
    }


def plot_machine_load_chart(analysis_results):
    makespan_value = analysis_results['makespan_value']
    machine_load_list = analysis_results['machine_load_list']
    num_machines_instance = analysis_results['num_machines']
    
    # --- 可視化1: 機械負荷の棒グラフ ---
    print("\n--- Visualization ---")
    plt.figure(figsize=(8, 5))
    machines = [f"Machine {m}" for m in range(num_machines_instance)]
    bars = plt.bar(machines, machine_load_list, color="skyblue", edgecolor="black")
    plt.axhline(
        makespan_value,
        color="red",
        linestyle="--",
        label=f"Makespan ({makespan_value:.2f})",
    )

    for bar in bars:
        yval = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            yval,
            f"{yval:.2f}",
            va="bottom",
            ha="center",
        )

    plt.xlabel("Machine")
    plt.ylabel("Total Processing Time (Load)")
    plt.title("Machine Load Distribution")
    plt.legend()
    plt.grid(axis="y", linestyle=":", alpha=0.7)
    plt.ylim(0, makespan_value * 1.15)
    plt.tight_layout()
    plt.show()


def plot_gantt_chart(analysis_results):
    assignment = analysis_results['assignment']
    makespan_value = analysis_results['makespan_value']
    job_times = analysis_results['job_times']
    num_machines_instance = analysis_results['num_machines']
    num_jobs_instance = len(job_times)
    
    # --- 可視化2: 割り当てのガントチャート ---
    # このチャートは各機械に割り当てられたジョブを順番に表示します。
    # 横軸は時間を表します。
    # ユーザー提供のスニペットのロジックに基づいています。

    fig, ax = plt.subplots(figsize=(12, max(4, num_machines_instance * 0.8)))
    # 各機械の現在の終了時間を追跡
    machine_end_times = np.zeros(num_machines_instance)
    job_colors = plt.cm.get_cmap("tab20", num_jobs_instance)  # ジョブごとに異なる色

    print("\nGenerating Gantt-style assignment chart...")

    # 機械とそれに割り当てられたジョブを反復処理
    for m_idx in range(num_machines_instance):
        # この機械に割り当てられたジョブを取得、一貫性のあるプロット用にソート（オプション）
        assigned_job_indices = sorted(assignment.get(m_idx, []))

        for job_idx in assigned_job_indices:
            if 0 <= job_idx < len(job_times):
                job_time = job_times[job_idx]
                start_plot_time = machine_end_times[m_idx]

                # ジョブのバーをプロット
                ax.barh(
                    m_idx,
                    job_time,
                    left=start_plot_time,
                    height=0.6,
                    align="center",
                    color=job_colors(job_idx % job_colors.N),
                    edgecolor="black",
                    alpha=0.8,
                )

                # バーの内側にジョブインデックステキストを追加
                # より良い視認性のためにテキスト位置をわずかに調整
                text_x = start_plot_time + job_time / 2.0
                text_y = m_idx  # バーの高さ内で垂直方向に中央揃え
                ax.text(
                    text_x,
                    text_y,
                    f"J{job_idx}",
                    va="center",
                    ha="center",
                    color="white",
                    fontweight="bold",
                    fontsize=9,
                )

                # この機械の終了時間を更新
                machine_end_times[m_idx] += job_time
            else:
                # 抽出が機能した場合、この条件は理想的には満たされない
                print(
                    f"Skipping plotting for invalid job index {job_idx} on machine {m_idx}"
                )

    # --- プロットの外観を設定 ---
    ax.set_xlabel("Execution Time")
    ax.set_ylabel("Machine Number")
    # y軸の目盛りを機械インデックスに合わせる
    ax.set_yticks(range(num_machines_instance))
    ax.set_yticklabels(
        [f"{m}" for m in range(num_machines_instance)]
    )  # 目盛りラベルを機械番号で表示
    ax.set_title("Gantt-Style Chart: Job Assignments per Machine")

    # 外観を改善
    ax.invert_yaxis()  # 機械0を上部に表示
    plt.grid(axis="x", linestyle=":", alpha=0.6)
    # 明確さのためにx軸の制限をメイクスパンよりわずかに超えて設定
    plt.xlim(0, makespan_value * 1.05)
    plt.tight_layout()
    plt.show()

## 8. ステップ1：小さなインスタンス

まず、PMSPの動作を理解するために小さなインスタンスから始めましょう。

### 8-1. インスタンスデータの準備

In [ ]:
# 例: それぞれの処理時間を持つ10個のジョブ
job_times_data = [5, 8, 3, 6, 9, 4, 7, 5, 2, 8]
# 例: 3台の機械が利用可能
num_machines_data = 3

# JijModelingプレースホルダに必要な辞書形式でデータを準備
instance_data = {
    "JT": job_times_data,
    "M": num_machines_data,
}

print("Instance Data:")
print(f"  Job Processing Times (JT): {instance_data['JT']}")
print(f"  Number of Machines (M): {instance_data['M']}")
print(f"  Number of Jobs (N derived): {len(instance_data['JT'])}")

### 8-2. JijModelingによるインスタンスの作成

In [ ]:
# 1. 特定のインスタンスデータで問題を評価するためのインタープリタを作成
interpreter = jm.Interpreter(instance_data)
# 2. 問題を評価してソルバー用の具体的なインスタンスを作成
instance = interpreter.eval_problem(problem)

### 8-3. JijZept Solverを使用したインスタンスの求解

定義されたモデルとインスタンスデータに対して最適解を見つけるため、JijZept Solverを使って求解します。

In [ ]:
# JijZept Solverをインポート
import jijzept_solver

# JijZept Solverを使用してインスタンスを求解
solution = jijzept_solver.solve(instance, time_limit_sec=5.0)

### 8-4. 結果の解釈と表示

次に、ソルバーから返された `solution` オブジェクトを処理して、最適なメイクスパンとジョブの割り当てを抽出します。

In [ ]:
# 結果を解析
results_small = analyze_schedule_results(solution, instance_data)

### 8-5. 結果の解析と可視化

ソリューションをさらに解析し、可視化しましょう。各機械の総処理時間（負荷）を計算し、最大負荷が最適化されたメイクスパンと一致することを確認します。

In [ ]:
# 機械負荷の棒グラフを表示
plot_machine_load_chart(results_small)

In [ ]:
# ガントチャートを表示
plot_gantt_chart(results_small)

## 9. ステップ2：大きなインスタンス

次に、モデルがどのようにスケールするかを確認するために、より大きなインスタンスでテストしてみましょう。

### 9-1. インスタンスデータの準備

In [ ]:
# 例: 300個のジョブ（10個のジョブを30回繰り返し）それぞれの処理時間
job_times_data_large = [5, 8, 3, 6, 9, 4, 7, 5, 2, 8] * 30
# 例: 90台の機械が利用可能
num_machines_data_large = 3 * 30

# JijModelingプレースホルダに必要な辞書形式でデータを準備
instance_data_large = {
    "JT": job_times_data_large,
    "M": num_machines_data_large,
}

print("Large Instance Data:")
print(f"  Job Processing Times (JT): {instance_data_large['JT'][:10]}... (showing first 10)")
print(f"  Number of Machines (M): {instance_data_large['M']}")
print(f"  Number of Jobs (N derived): {len(instance_data_large['JT'])}")

### 9-2. JijModelingによるインスタンスの作成

In [ ]:
# 1. 特定のインスタンスデータで問題を評価するためのインタープリタを作成
interpreter_large = jm.Interpreter(instance_data_large)
# 2. 問題を評価してソルバー用の具体的なインスタンスを作成
instance_large = interpreter_large.eval_problem(problem)

### 9-3. SCIPを使用したインスタンスの求解（OMMXPySCIPOptAdapter）

ここでは、ommx-pyscipopt-adapterパッケージによって提供されるSCIPを使用します。

In [ ]:
# 必要なアダプターをインポート
from ommx_pyscipopt_adapter import OMMXPySCIPOptAdapter

# OMMX SCIPアダプターを使用してインスタンスを求解
adapter = OMMXPySCIPOptAdapter(instance_large)
model = adapter.solver_input
model.setParam("limits/time", 5.0)
model.optimize()
solution_large_scip = adapter.decode(model)

In [ ]:
# 結果を解析
results_large_scip = analyze_schedule_results(solution_large_scip, instance_data_large)

In [ ]:
# 機械負荷の棒グラフを表示
plot_machine_load_chart(results_large_scip)

In [ ]:
# ガントチャートを表示
plot_gantt_chart(results_large_scip)

### 9-4. JijZept Solverを使用したインスタンスの求解

次に、JijZept Solverを使用します。

In [ ]:
# JijZept Solverを使用してインスタンスを求解
solution_large_jijzept_solver = jijzept_solver.solve(instance_large, time_limit_sec=5.0)

In [ ]:
# 結果を解析
results_large_jijzept_solver = analyze_schedule_results(solution_large_jijzept_solver, instance_data_large)

In [ ]:
# 機械負荷の棒グラフを表示
plot_machine_load_chart(results_large_jijzept_solver)

In [ ]:
# ガントチャートを表示
plot_gantt_chart(results_large_jijzept_solver)

## 10. まとめ

このノートブックでは、**並列機械スケジューリング問題（PMSP）** をモデル化し、解決する方法を示しました。

* 問題を定義しました：独立したジョブを同一の並列機械に割り当てて、メイクスパン（最大機械完了時間）を最小化します。
* JijModelingを使用して数理モデルを定式化し、割り当て用のバイナリ変数 `x[i, m]` と目的用の連続変数 `makespan` を定義しました。
* 各ジョブが正確に1回割り当てられることを保証し、メイクスパンが任意の機械の最大負荷を正しく反映することを保証する制約を含めました。
* 特定のインスタンスデータ（ジョブ処理時間、機械数）を提供しました。
* 最後に、ソルバーのソリューションオブジェクトから結果を抽出して解釈し、計算された機械負荷に対してメイクスパンを検証し、プロットを使用して割り当てと負荷を可視化しました。

このプロセスは、JijModelingのような数理最適化ライブラリをJijZept Solverと組み合わせることで、実用的なスケジューリングの課題に取り組む方法を示しています。同じ方法を、より複雑な問題に拡張することができます。